In [1]:
import requests
from bs4 import BeautifulSoup
import subprocess
import os
import shutil

In [2]:
DOWNLOAD_BASE_PATH = "D:/PhD/Model_Building/APKs_and_GitRepos"

In [3]:
def get_menu_links():
    url_to_call = "https://fossdroid.com/"

    response = requests.get(url_to_call, headers={'User-Agent': 'Mozilla/5.0'})
    response_code = response.status_code

    if response_code != 200:
        print("Some Error")
        return

    html_content = response.content
    dom = BeautifulSoup(html_content, 'html.parser')
    all_menu_content = dom.select("a.mdl-navigation__link")

    menu_links = []

    for each_menu_content in all_menu_content:
        href_link = each_menu_content.attrs["href"]

        next_link = url_to_call+href_link[1:]+"whats_new.html"

        menu_links.append(next_link)

    return menu_links

In [4]:
def get_view_app_links(all_menu_links):
    page_links = []
    for each_menu_link in all_menu_links:
        response = requests.get(each_menu_link, headers={'User-Agent': 'Mozilla/5.0'})
        response_code = response.status_code

        if response_code != 200:
            print("Some Error")
            return

        html_content = response.content
        dom = BeautifulSoup(html_content, 'html.parser')
        all_view_app_content = dom.select("div.mdl-card__title h5")

        for each_view_app_content in all_view_app_content:

            href_link = each_view_app_content.a.attrs["href"]
            page_link = "https://fossdroid.com" + href_link

            page_links.append(page_link)

    return page_links

In [5]:
def download_one_by_one_apk_git(all_links,visiting_links_to,visiting_links_from):      
        
    if(len(all_links) >= visiting_links_from):
        
        if(len(all_links)<visiting_links_to):
            visiting_links_to = len(all_links)
            
        for i in range(visiting_links_from,visiting_links_to):
            try:
                os.chdir(DOWNLOAD_BASE_PATH) 
                response = requests.get(all_links[i], headers={'User-Agent': 'Mozilla/5.0'})
                response_code = response.status_code
                if response_code != 200:
                    print("Some Error")
                    return


                html_content = response.content
                dom = BeautifulSoup(html_content, 'html.parser')
                each_page_contents_apk = dom.select("#fd-download-app")        

                apk_name ="" 
                is_already_downloaded = False
                for each_page_content_apk in each_page_contents_apk:
                    href_link = each_page_content_apk.attrs["href"]
                    apk_link = "https://fossdroid.com"+href_link                  

                    r = requests.get(apk_link, allow_redirects=True)
                    filename = apk_link[24:]
                    apk_name = filename[:len(filename)-4]
                    print(apk_name)                
                    if (os.path.isdir(apk_name)!=True):     
                        is_already_downloaded = False
                        os.mkdir(apk_name)
                        os.chdir(DOWNLOAD_BASE_PATH+"/"+apk_name)
                        open(filename, 'wb').write(r.content)
                        
                    else:
                        is_already_downloaded = True                     
                
                    
                if is_already_downloaded == False:
                    each_page_contents_git = dom.select("#fd-source-app")   
                    for each_page_content_git in each_page_contents_git:
                        github_link = each_page_content_git.attrs["href"]
                        subprocess.call("git clone "+github_link+".git")
                    
            except Exception as e:
                    print(e)
                    continue 
                

In [6]:
def delete_git_unavailable_folders():
    git_unavailable_list = []
    folders = os.listdir(DOWNLOAD_BASE_PATH)
    print("Number of all downloads : ", len(folders))

    for folder in folders:
        try:
            files  = os.listdir(DOWNLOAD_BASE_PATH+"/"+folder)
            if(len(files)<2):
                git_unavailable_list.append(folder)
        except:
            continue    

    for non_git_folder in git_unavailable_list:
        try:
            shutil.rmtree(DOWNLOAD_BASE_PATH+"/"+non_git_folder)
        except:
            continue
            
    print ("Number of finalised Downloads : ", len(folders) - len(git_unavailable_list))

In [7]:
def execute_process():        
    print("****************Process Started*****************")
    
    print("****************Started Scraping****************")

    print("***********Retrieveing all menu links***********")
    all_menu_links = get_menu_links()
    
    menu_count = len(all_menu_links)
    print("Total number of menu :",menu_count)
    
    for menu_no in range(0, menu_count):             
    #for menu_no in range(8, 9): // enable to customize the search
        try:         
            print("**************Retrieveing all links**************")
            all_links = get_view_app_links(all_menu_links[menu_no:menu_no+1])
            all_links_count = len(all_links)
            print("Total number of links :",all_links_count)
                    
            for visiting_links_from in range (0, all_links_count, 20):    
            #for visiting_links_from in range (80, 100, 20): // enable to customize the search
                try:
                    visiting_links_to = (visiting_links_from+20)
                    print("Menu no = ",menu_no, " Visiting links_from = ",visiting_links_from, " Vsiting links_to = ",visiting_links_to)   
                    print("****************Downloading files****************")
                    download_one_by_one_apk_git(all_links,visiting_links_to,visiting_links_from)
                except Exception as e:
                    print(e)
                    continue 
                        
        except Exception as ex:
            print(ex)
            continue     
    
    print("***********Cleaning GitHub repositories unavaialble downloads***********")
    
    delete_git_unavailable_folders()
                
    print("****************Process Completed****************")    

In [8]:
execute_process()

****************Process Started*****************
****************Started Scraping****************
***********Retrieveing all menu links***********
Total number of menu : 17
**************Retrieveing all links**************
Total number of links : 519
Menu no =  0  Visiting links_from =  0  Vsiting links_to =  20
****************Downloading files****************
hail
network-2
simple-keyboard-2
freezeyou
simple-clipboard-editor
android-2-linux-notifications
upgradeall
wyfy
clipeus
volta
screen-time
scrunch
adbio
hwloc-lstopo
adsilence
gcam-services-provider-basic
defo-ech-apps
mergedwifinlp
save-on-device
autodnd
Menu no =  0  Visiting links_from =  20  Vsiting links_to =  40
****************Downloading files****************
bluepass
ttime
droidify
search-to-browser
activity-manager
easyphone
link-eye
sithakuru
volumelockr
fdroid-nearby
extirpater
prepaid-balance
usb-gadget-tool
leon-the-url-cleaner-beta
watt
smart-autoclicker
camera-folder
scoop
busybox-installer
pddroid-party
Menu no 

mmskeeper
netcounter
websearch
random-app
katana
fast-brightness-control-widget
open-flashlight
brightness-widget
batterybot-pro
freeotp
linconnect
silent-night
preferences-manager
color-namer
mounts2sd
performance-control
minutes-checker-for-virgin-mobile
apps2org
kde-connect
Menu no =  0  Visiting links_from =  420  Vsiting links_to =  440
****************Downloading files****************
ghost-commander-sftp-plugin
ghost-commander-samba-plugin
android-token
fonbot
maxs-main
maxs-module-smsread
maxs-module-smssend
maxs-transport-xmpp
maxs-module-fileread
maxs-module-smswrite
maxs-module-bluetooth
maxs-module-clipboard
maxs-module-filewrite
maxs-module-smsnotify
maxs-module-ringermode
maxs-module-contactsread
maxs-module-bluetoothadmin
maxs-module-phonestateread
list-my-apps
launch-app-ops
Menu no =  0  Visiting links_from =  440  Vsiting links_to =  460
****************Downloading files****************
kolab-client
root-verifier
oandbackup
usb-device-info
droid-examiner
clipboard-bea

lnkshortener
inventum
behe-explorer
k4android
kiba
teleposter
twdwyr
dolidroid
Menu no =  1  Visiting links_from =  320  Vsiting links_to =  340
****************Downloading files****************
browser
simple-irc
good-weather
materialfbook
slimsocial-for-twitter
slimsocial-for-facebook
toffeed
dns66
libretorrent
sigfood
materialistic
inbox-pager
simplert
commons
netcfg-widget
viborafeed
url-forwarder
fbclient
jumpgo
afh-downloader
Menu no =  1  Visiting links_from =  340  Vsiting links_to =  360
****************Downloading files****************
opengapps-downloader
qing-xin-tian-qi
nextcloud
xposed-downloader
nextcloud-beta
watransmitter
messenger-bypasser
pjuu-2
bienvenido-a-internet
basketbuild-downloader
tensor
privacy-browser
diaspora-2
getchromium
wetter
imapnotes2
bulkshare-2
sg-for-steamgifts
facebook-notifications
open-link-with
Menu no =  1  Visiting links_from =  360  Vsiting links_to =  380
****************Downloading files****************
beta-updater-for-whatsapp
voice-re

puzzle
piratedice
spacepeng
edtool
freeminer
minetest
flickit
solitairecg
paddletennis
mh4u-database
osrs-helper
card-game-scores
love-game
ogame-on-android
dicer
signal-from-mars
motac
igo
dotrace
hungarian-rings-for-android
Menu no =  2  Visiting links_from =  260  Vsiting links_to =  280
****************Downloading files****************
number-guesser
score-tarot
deckerino
shattered-pixel-dungeon
hangman
monster-hunter-3u-database
recursive-runner
dots-squares
pixel-dungeon
marble-one
gobandroid
chess
sudoq
stk-addon-viewer
guess
minesweeper
chesswatch
boardgamegeek
24game
retro-breaker
Menu no =  2  Visiting links_from =  280  Vsiting links_to =  300
****************Downloading files****************
exalted-dicer
liquid-wars-os
toe
dragongoapp
2048
catan-dice-game
isolmoa
wot-tank-quiz
heriswap
easy-dice
oware
gomoku
sudowars
text-fiction
puzzles
papanikolis-submarine
geometri-destroyer
masterwork-tools-pathfinder-open-reference
clash-of-balls
adosbox
Menu no =  2  Visiting links_f

eyecam
rainwave
andof
presentation
espeak-tts
galbijjim-searcher
clementine-remote
product-open-data
droidupnp
Menu no =  3  Visiting links_from =  300  Vsiting links_to =  320
****************Downloading files****************
amplayer
prestissimo
quicksnap
muspy-for-android
spydroid
nachtlager-downloader
wifi-remote-play
foocam-beta
yaacc
droidgain
melodeon
accordion
controldlna
floating-image
music
svg-viewer
micdroid
wifi-camera
episodes
camtimer
Menu no =  3  Visiting links_from =  320  Vsiting links_to =  340
****************Downloading files****************
picard-barcode-scanner
nettts
asciicam
dsub
kaleidoscope
sonorox
vlc
apollo
andless
hdhomerun-signal-meter
slider
tuner
teacup
mobilewebcam
squeezer
midisheetmusic
just-player
ardroid
sickstache
barcode-scanner
Menu no =  3  Visiting links_from =  340  Vsiting links_to =  360
****************Downloading files****************
vanilla-music
hexiano
gmote
scrobble-droid
simple-lastfm-scrobbler
antennapod
jupiter-broadcasting
gcst

v2ray
shadowsocks-foss
spider
public-ip
androbd-sensorprovider
ble-peripheral-simulator
savapage-print-service
androbd-gpsprovider
card10-companion
ipfs
calliope-mini
selfnet-wifi-setup
Menu no =  5  Visiting links_from =  80  Vsiting links_to =  100
****************Downloading files****************
share-to-computer
badge-magic
actualizador-odns
hue-widgets
android-samba-client
controlloid
openpods
my-leaf
crazyflie-client
pihole-droid
multivnc
presence-publisher
tpmsmonitor
notify2jabber
35c3-wifi-setup
panictrigger
pigrid-companion
aruba-networks-login
beamcontrol-for-smart-interface
lws
Menu no =  5  Visiting links_from =  100  Vsiting links_to =  120
****************Downloading files****************
lws-qr
manyverse
sleepywifi
yoke
fit-notifications
briar
ambitsync
dreamdroid
mqttpublisher-plugin
nitroshare
vpn-hotspot
alsa-mixer-webui
netswitch
evecontrol-2
tuc-wlan
syncplayer
tinc-2
trebleshot
moscow-wifi-autologin
dapnet
Menu no =  5  Visiting links_from =  120  Vsiting links_t

anysoftkeyboard-brazilian
diary-2
bierverkostung
Menu no =  7  Visiting links_from =  80  Vsiting links_to =  100
****************Downloading files****************
anysoftkeyboard-hebrew-2-3
snotepad
just-notes
notebuddy
sealnote
anysoftkeyboard-tatar
orgzly
note-crypt-pro
anysoftkeyboard-russkij
omni-notes-foss
anysoftkeyboard-finnish
anysoftkeyboard-italian
anysoftkeyboard-latvian
anysoftkeyboard-slovene
anysoftkeyboard-basque
anysoftkeyboard-norwegian
anysoftkeyboard-swedish
anysoftkeyboard-german
anysoftkeyboard-hebrew-2
syncorg
Menu no =  7  Visiting links_from =  100  Vsiting links_to =  120
****************Downloading files****************
memento
notes
todo-list
shopping-list
pdf-creator
padland
slide
packlist
notepad-2
oi-shopping-list
unote
poet-assistant
anysoftkeyboard-neo2
owncloud-notes
notification-notes
minimal
shoppinglist
counter
kolab-notes
writeily-pro-2
Menu no =  7  Visiting links_from =  120  Vsiting links_to =  140
****************Downloading files**************

document-viewer
fbreader-calibre-connector
tibetan-pad
acv
hn
comics-reader
japanese-name-converter
oschina
zandy
newsblur
bites
epubator
fbreader-tts+-plugin
fbreader-tts-plugin
adsdroid
scanner-for-zotero
android-tipitaka
lampshade
and-bible
Menu no =  9  Visiting links_from =  140  Vsiting links_to =  160
****************Downloading files****************
apv-pdf-viewer
pageturner
vudroid
arxiv-mobile
bookworm
fbreader
cool-reader
opendocument-reader
**************Retrieveing all links**************
Total number of links : 147
Menu no =  10  Visiting links_from =  0  Vsiting links_to =  20
****************Downloading files****************
dark-theme
doodle-live-wallpapers
arcticons-light
dark-mode-live-wallpaper
pie-launcher
text-launcher
mondstern-acrylic-icons
sentien-launcher
blue-line-console
arcticons
materials-live-wallpaper
gif-live-wallpaper
olauncher
pixiv-for-muzei-3
posidon-launcher
circo
vectorify-da-home
s60-icon-pack
bliss-launcher
wikimedia-commons-for-muzei-2
Menu no 

man-man
alogcat-root
ndef-tools-for-android
gears
ajsha-android-java-shell-app
ttl-editor
bf-interpreter
yaft
travis-jr
rot13-translator
bluetooth-terminal
java-installer
dotty
arduino-uno-communicator
balanduino
bluetooth-viewer-lite
gogodroid
debian-kit
servdroid
urlazy
Menu no =  12  Visiting links_from =  100  Vsiting links_to =  120
****************Downloading files****************
debiandroid
cooper
pylauncher
adb-manager
mobiperf
stweaks
null-launcher
android-usb-serial-monitor-lite
hosts-editor
device-frame-generator
nofrills-cpu-control-classic
andlytics
intent-intercept
charmap
dalvik-explorer
vi-improved-touch
buildprop-editor
cowsay
urforms-database
oasvn-pro
Menu no =  12  Visiting links_from =  120  Vsiting links_to =  140
****************Downloading files****************
octodroid
android-resources
catlog
agrep
sensor-readout
textwarrior
voodoo-screen-test-patterns
screeninfo
network-log
ministro
httpmon
nstools
920-editor
cpan-sidekick
github
unicode-map
terminal-emulat

freedcam
pixel-artist
monochromatic
swpieview
quick-draw-everywhere
velocity-calculator
pixelflood
memetastic
barcodegen
tuxpaint
meme-creator
deadpixel
effects-pro
micopi
acrylic-paint
defcol
gfxtablet
ocr-test
send-reduced
rage-maker
Menu no =  16  Visiting links_from =  40  Vsiting links_to =  60
****************Downloading files****************
memopad
scribbler
quill
pixelesque
droid-draw
coloring-for-kids
markers
andross
***********Cleaning GitHub repositories unavaialble downloads***********
Number of all downloads :  3775
Number of finalised Downloads :  3237
****************Process Completed****************
